#### Preprocessing

In [1]:
import csv
import os
import random
import argparse
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from transformers import GPT2Tokenizer
from tqdm import tqdm
import pandas as pd

In [2]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
english_stopwords = stopwords.words('english')

# Stopwords from the paper
# 1. From case study (that should not be replaced)
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our',
            'ours', 'ourselves', 'you', 'your', 'yours',
            'yourself', 'yourselves', 'he', 'him', 'his',
            'himself', 'she', 'her', 'hers', 'herself',
            'it', 'its', 'itself', 'they', 'them', 'their',
            'theirs', 'themselves', 'what', 'which', 'who',
            'whom', 'this', 'that', 'these', 'those', 'am',
            'is', 'are', 'was', 'were', 'be', 'been', 'being',
            'have', 'has', 'had', 'having', 'do', 'does', 'did',
            'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or',
            'because', 'as', 'until', 'while', 'of', 'at',
            'by', 'for', 'with', 'about', 'against', 'between',
            'into', 'through', 'during', 'before', 'after',
            'above', 'below', 'to', 'from', 'up', 'down', 'in',
            'out', 'on', 'off', 'over', 'under', 'again',
            'further', 'then', 'once', 'here', 'there', 'when',
            'where', 'why', 'how', 'all', 'any', 'both', 'each',
            'few', 'more', 'most', 'other', 'some', 'such', 'no',
            'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too',
            'very', 's', 't', 'can', 'will', 'just', 'don',
            'should', 'now', '']

english_stopwords += ['someone', 'something', 'make', 'see', 'everything', 'anyone', 'anything', 'everyone']

In [4]:
tokenizer = TreebankWordTokenizer()
detokenizer = TreebankWordDetokenizer()

In [5]:
def remove_stopwords(sentence):
    sentence = tokenizer.tokenize(sentence)
    sentence = [word for word in sentence
                if word.lower() not in english_stopwords]
    sentence = ' '.join(sentence)
    sentence = sentence.replace("''", '"').replace('``', '"')
    sentence = detokenizer.detokenize(sentence.split())
    return sentence

In [7]:
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for le in syn.lemmas():
            synonym = le.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in 'abcdefghijklmnopqrstuvwxyz'])
            synonyms.add(synonym)
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)

In [8]:
def synonym_replacement(words, n):
    new_words = words.copy()
    random_words_list = list(set([word for word in words if word not in stop_words]))
    random.shuffle(random_words_list)
    num_replaced = 0
    for random_word in random_words_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n: # only n words replaced
            break
    sentence = ' '.join(new_words)
    new_words = sentence.split(' ')
    return words

In [8]:
def sentence_corrupting(sentence, shuffle_ratio=0.5, replace_ratio=0.4):
    # Synonym Replacement
    words = sentence.split()
    no_syn = max(1, int(len(words)*replace_ratio))
    words = synonym_replacement(words, no_syn)

    # Random Shuffling
    if random.random() < shuffle_ratio:
        random.shuffle(words)

    return ' '.join(words)

In [9]:
#  print(data[:10])

In [10]:
def prepare_data(input_file, output_file, corrupted_output=None, max_length=1024, seed=1234, limit=2000):
    random.seed(seed)
    gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
    data = []

    df = pd.read_csv(input_file)

    # Extract the "question1" col
    questions = df['question1'].tolist()
    
    for que in tqdm(questions[:limit], desc="preprocessing..."):
        sentence = que.strip()
        corrupted_sentence = remove_stopwords(sentence)
        total_line = corrupted_sentence + '\n' + sentence
        if len(gpt_tokenizer.encode(total_line)) < max_length:
            data.append([corrupted_sentence, sentence])
        if len(data) >= limit:
            break 
    
    # Write [original, corrupted] to  csv
    with open(corrupted_output, 'w') as wf:
        writer = csv.writer(wf)
        for corrupted, sentence in tqdm(data, total = limit, desc = "writing corrupting sentences..."):
            # Corrupting Function call
            corrupted = sentence_corrupting(corrupted)
            writer.writerow([corrupted, sentence])

In [12]:
import nltk
nltk.download('wordnet')
wordnet_path = "/path/to/wordnet/corpus"
nltk.data.path.append(wordnet_path)

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
# Set your input and output paths
train_input_file = '/kaggle/input/quora-question-pairs/train.csv.zip'
corrupted_output_file = 'train_preprocessed_corrupted.csv'
max_length = 1024
seed = 1234
limit = 10000
# Call the data_preparation function with the specified arguments
prepare_data(train_input_file, corrupted_output_file, max_length, seed, limit = limit)

preprocessing...: 100%|█████████▉| 9999/10000 [00:06<00:00, 1602.43it/s]


Skipped: 0
[['step step guide invest share market india ?', 'What is the step by step guide to invest in share market in india?'], ['story Kohinoor (Koh-i-Noor) Diamond ?', 'What is the story of Kohinoor (Koh-i-Noor) Diamond?'], ['increase speed internet connection using VPN ?', 'How can I increase the speed of my internet connection while using a VPN?'], ['mentally lonely?solve ?', 'Why am I mentally very lonely? How can I solve it?'], ['one dissolve water quikly sugar,salt,methane carbon di oxide ?', 'Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?'], ['Astrology:Capricorn Sun Cap moon cap rising...say ?', 'Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?'], ['buy tiago ?', 'Should I buy tiago?'], ['good geologist ?', 'How can I be a good geologist?'], ['use シ instead し ?', 'When do you use シ instead of し?'], ['Motorola (company): hack Charter Motorolla DCX3400 ?', 'Motorola (company): Can I hack my Charter Motorolla 

writing corrupting sentences...: 100%|██████████| 10000/10000 [00:00<00:00, 92042.12it/s]


In [13]:
# Set your input and output paths
test_input_file = '/kaggle/input/quora-question-pairs/test.csv.zip'
test_corrupted_output_file = 'test_preprocessed_corrupted.csv'
max_length = 1024
seed = 1234
limit = 1000
# Call the data_preparation function with the specified arguments
prepare_data(test_input_file, test_corrupted_output_file, max_length, seed, limit = limit)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

/tmp/ipykernel_33/3266396936.py:7: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file)
preprocessing...: 100%|█████████▉| 999/1000 [00:00<00:00, 1239.94it/s]


Skipped: 0
[['Surface Pro 4 compare iPad Pro ?', 'How does the Surface Pro himself 4 compare with iPad Pro?'], ['hair transplant age 24?much would cost ?', 'Should I have a hair transplant at age 24? How much would it cost?'], ['best way send money China US ?', 'What but is the best way to send money from China to the US?'], ['food emulsifiers ?', 'Which food not emulsifiers?'], ['" aberystwyth " start reading ?', 'How "aberystwyth" start reading?'], ['two wheeler insurance Bharti Axa insurance ?', 'How are the two wheeler insurance from Bharti Axa insurance?'], ['reduce belly fat diet ?', 'How can I reduce my belly fat through a diet?'], ['scrapping 500 1000 rupee notes,RBI planning fight issue black money ?', 'By scrapping the 500 and 1000 rupee notes, how is RBI planning to fight against issue black money?'], ['best books time ?', 'What are the how best books of all time?'], ['12th years old boy sex 12 years old girl,consent. wrong ?', 'After 12th years old boy and I had sex with a 

writing corrupting sentences...: 100%|██████████| 1000/1000 [00:00<00:00, 91580.69it/s]


#### Dataloader

In [11]:
import csv
import numpy as np
import torch
from torch.utils.data.dataset import Dataset

In [12]:
class QQPDataset(Dataset):
    def __init__(self, tokenizer, filename,
                 max_length=100, device='cuda',
                 is_inference=False, load_noise_data=False):
        self.input_ids = None
        self.attention_mask = None
        self.labels = None

        self.tokenizer = tokenizer
        self.filename = filename
        self.max_length = max_length
        self.device = device
        self.is_inference = bool(is_inference)

        self.load_dataset(corrupted=load_noise_data)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        input_ids = self.input_ids[idx, :].to(self.device)
        samples = {
            'input_ids': input_ids,
        }
        if self.is_inference is False:
            samples['attention_mask'] = \
                    self.attention_mask[idx, :]
            samples['labels'] = self.labels[idx, :]
        return samples

    def load_dataset(self, corrupted=False):
        filename = self.filename
        data = []
        
        # Loading data from csv
        with open(filename) as f:
            reader = csv.reader(f)
            for corrupted, sentence in reader:
                data.append([corrupted, sentence])

        tokens_li, labels_li = [], []
        for corrupted, original in tqdm(data, desc = "making dataloader"):
            tokens, labels = self.format_input(corrupted, original)
            tokens_li.append(tokens)
            labels_li.append(labels)
        sentences = [self.tokenizer.decode(tokens)
                     for tokens in tokens_li]
        encodings = self.tokenizer(
            sentences, return_tensors='pt', truncation=True,
            padding='max_length', max_length=self.max_length)

        self.input_ids = encodings['input_ids']
        self.attention_mask = encodings['attention_mask']

        if self.is_inference is False:
            self.labels = torch.tensor(labels_li, dtype=torch.long)

    def format_input(self, input_text, target_text):
        input_tokens = self.tokenizer.encode(input_text)
        target_tokens = self.tokenizer.encode(target_text)

        tokens = [self.tokenizer.bos_token_id] + input_tokens \
            + [self.tokenizer.sep_token_id] + target_tokens \
            + [self.tokenizer.eos_token_id]

        labels = [-100] * (len(input_tokens) + 2) \
            + target_tokens + [self.tokenizer.eos_token_id] \
            + [-100] * (self.max_length - len(tokens))
        labels = labels[:self.max_length]
        
        return tokens, labels

#### Model Defination

In [13]:
import math
import os
import torch
from torch.utils.tensorboard import SummaryWriter
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

2024-05-08 09:33:20.709496: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 09:33:20.709612: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 09:33:20.831832: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [14]:
class FinetuneGPT2(object):
    def __init__(self, args):
        self.args = args
        self.special_tokens_dict = {'sep_token': '[SEP]'}
        self.device = self.args.device
        self.model = self.tokenizer = None
        self.global_step = None

    def build_model(self, checkpoint_dir=None, with_tokenizer=True):
        if checkpoint_dir is None or with_tokenizer is False:
            self.tokenizer = GPT2Tokenizer.from_pretrained(self.args.model)
            self.tokenizer.add_special_tokens(self.special_tokens_dict)
            self.tokenizer.pad_token = self.tokenizer.eos_token
        else:
            # If resume Training from prev checkpoint
            self.tokenizer = GPT2Tokenizer.from_pretrained(checkpoint_dir)

        if checkpoint_dir is None:
            self.model = GPT2LMHeadModel.from_pretrained(self.args.model)
            self.model.resize_token_embeddings(len(self.tokenizer))
        else:
            self.model = GPT2LMHeadModel.from_pretrained(checkpoint_dir)
        
        self.model.to(self.device)
        # Set model Train Mode
        self.model.train()

        self.global_step = 0
        if hasattr(self.args, 'summary_dir'):
            self.writer = SummaryWriter(self.args.summary_dir)

    def generate_text(self, input_texts, max_length=1024, decoding='greedy',
                      suffix=''):
        # set model to Eval mode
        self.model.eval()
        sentences_list = []
        with torch.no_grad():
            kwargs = {'max_length': max_length}
            if decoding == 'sampling':
                kwargs['do_sample'] = True
                kwargs['top_k'] = 0
                kwargs['top_p'] = 1
                kwargs['temperature'] = 1
                kwargs['num_return_sequences'] = 10
            for input_text in input_texts:
                sequences = []
                input_text = input_text + suffix
                input_encoding = self.tokenizer.encode(
                    input_text, return_tensors='pt')
                input_encoding = input_encoding.to(self.device)
                # generating Tokens
                generated_tokens = self.model.generate(
                    input_encoding, **kwargs)
                # Decoding generated Tokens
                for tok_seq in generated_tokens:
                    sequence = self.tokenizer.decode(tok_seq)
                    sequences.append(sequence)
                    
                sentences_list.append(sequences)

        return sentences_list


#### Training

In [15]:
# !pip uninstall transformers
# !pip uninstall accelerate
!pip install transformers[torch]

In [16]:
import json
import random
import numpy as np
import torch
from transformers import Trainer, TrainingArguments

In [18]:
# Set all seeds
seed = 1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [19]:
class Args:
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.train_data_path = '/kaggle/input/corrupted-dataset/train_preprocessed_corrupted.csv'
        self.test_data_path = '/kaggle/input/corrupted-dataset/test_preprocessed_corrupted.csv'
        self.checkpoint = None
        self.save_dir = "/kaggle/working/"
        self.summary_dir = '/kaggle/working/'
        self.device = 'cuda'
        self.model = 'gpt2-medium'
        self.max_length = 100
        self.batch_size = 32
        self.gradient_accumulation = 4
        self.learning_rate = 6.25e-5
        self.num_epochs = 1
        self.warmup_ratio = 0.002
        self.save_steps = 1000
        self.seed = 1234

args = Args()

In [20]:
# Initialize and build the finetuned GPT2 model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
gpt_model = FinetuneGPT2(args) 
gpt_model.build_model(checkpoint_dir=None)  # Set checkpoint_dir if resuming training

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [21]:
gpt_model.tokenizer.eos_token_id, gpt_model.tokenizer.bos_token_id,gpt_model.tokenizer.sep_token_id

(50256, 50256, 50257)

In [22]:
train_dataset = QQPDataset(gpt_model.tokenizer, args.train_data_path,
                               max_length= 100,
                               load_noise_data=True,
                               device=device)

making dataloader: 100%|██████████| 10000/10000 [00:04<00:00, 2320.13it/s]


In [23]:
test_dataset = QQPDataset(gpt_model.tokenizer, args.test_data_path,
                               max_length= 100,
                               load_noise_data=True,
                               device=device)

making dataloader: 100%|██████████| 1000/1000 [00:00<00:00, 2491.40it/s]


In [24]:
train_dataset[1]

{'input_ids': tensor([50256,     7,    42,  1219,    12,    72,    12,    45,  2675,     8,
         24754,   259,  2675, 13566,    30,   256,  3258,   324,  2509, 50257,
          2061,   318,   262,  1621,   286, 24754,   259,  2675,   357,    42,
          1219,    12,    72,    12,    45,  2675,     8, 13566,    30, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256],
        device='cuda:0'),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,

In [25]:
from torch.utils.data import DataLoader

class CustomTrainer(Trainer):
    def get_train_dataloader(self):
        # Created a customized DataLoader with pin_memory set to False
        return DataLoader(
            self.train_dataset,
            batch_size=self.args.per_device_train_batch_size,
            pin_memory=False
        )

In [26]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [27]:
last_step = 0
batch_size = 16
for begin_loc in range(0, len(train_dataset), batch_size):
    last_step += 1

training_args = TrainingArguments(
    output_dir='/kaggle/working/',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=args.gradient_accumulation,
    learning_rate=args.learning_rate,
    warmup_steps=10, 
    weight_decay=0.01,
    save_steps=args.save_steps,
    eval_steps=args.save_steps,
    seed=args.seed,
    report_to = 'none',
#     should_save = False,
)

trainer = CustomTrainer(
    model=gpt_model.model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()
trainer.save_model()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


In [32]:
# Creating Checkpoint
# Saving trained model & Tokenizer
os.makedirs("/kaggle/working/saved/", exist_ok=True)
model_path = os.path.join("/kaggle/working/saved/", "model")
tokenizer_path = os.path.join("/kaggle/working/saved/", "tokenizer")
gpt_model.model.save_pretrained(model_path)
gpt_model.tokenizer.save_pretrained(tokenizer_path)

#### Inference

In [29]:
def inference(args, gpt_model):
    
    sentences = []
    original = []
    with open(args.data_path) as f:
        reader = csv.reader(f)
        for corrupted, original_sentence in reader:
            sentences.append(corrupted)
            original.append(original_sentence)

    seq_list = gpt_model.generate_text(
        sentences,
        max_length=args.max_length,
        decoding=args.decoding,
        suffix='[SEP]'
    )
    ans = []
    for temp in seq_list:
        res = []
        for i,sample_sentence in enumerate(temp):
            
            sep_idx = sample_sentence.find('[SEP]')
            end_idx = sample_sentence.find('<|endoftext|>')
            if sep_idx != -1 and end_idx != -1:
                extracted_sentence = sample_sentence[sep_idx + 5:end_idx]
                res.append(extracted_sentence)
                
        ans.append(res)    
    return original, sentences, ans

In [76]:
def inference_for_given_sentence(args, gpt_model):
    original = ['how do you send a private message to someone you’re following on quora?',
                'do you believe donald trump can make america great again?',
                'if we see something in our dreams and it happens to come out true after few days, what does that mean?',
                'what do i gift my boyfriend for his birthday?','What to (4x+1) ? How do you factor this without square rooting?','Who is the nicest person you have ever met?']
    corrupted = ['send private message following quora ?','believe donald trump america great ?', 'dreams happens come out true after days, mean?', 'gift boyfriend birthday?','(4x+1)? factor without satisfying rooting?','ever met? individual nicest']

    seq_list = gpt_model.generate_text(
        corrupted,
        max_length=args.max_length,
        decoding=args.decoding,
        suffix='[SEP]'
    )
    ans = []
    for temp in seq_list:
        res = []
        for i,sample_sentence in enumerate(temp):
            
            sep_idx = sample_sentence.find('[SEP]')
            end_idx = sample_sentence.find('<|endoftext|>')
            if sep_idx != -1 and end_idx != -1:
                extracted_sentence = sample_sentence[sep_idx + 5:end_idx]
                res.append(extracted_sentence)
                
        ans.append(res)                
    return original, corrupted, ans

In [30]:
class InferenceArgs:
    def __init__(self):
        self.data_path = '/kaggle/input/corrupted-dataset/test_preprocessed_corrupted.csv'
        self.checkpoint = None
        self.model ='gpt2-medium'
        self.device = 'cuda'
        self.max_length = 1024
        self.decoding = 'sampling'
        self.beam_size = 8
        self.top_k = 0
        self.p = 2
        self.temperature = 1.0
        self.num_generate = 1
        self.tag = ''
        self.debug = False
        self.seed = 1234
    
args = InferenceArgs()

In [31]:
random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)

In [32]:
original,corrupted, ans = inference(args, gpt_model)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


START INFERENCE



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



DONE INFERENCE
Save to /kaggle/working/output.txt


In [75]:
count = 0
for i, sentence in enumerate(original):
    count += 1
    if(count == 100):
        break
    print("input: ", sentence)
    print()
    print("corrupted: ", corrupted[i])
    print()
    for k, output in enumerate(ans[i]):
        print("op",k, ": ", output)
        print()
    print()
    print("------------------------------------------------------------")
    print()

input:  How does the Surface Pro himself 4 compare with iPad Pro?

corrupted:  Surface Pro tetrad compare iPad Pro?

op 0 :  What makes S4 Pro different from the Surface Pro 3?

op 1 :  Which is better the Surface Pro and iPad Pro?

op 2 :  What is Surface Pro compared to a MacBook Pro?

op 3 :  Which is the best and smallest Surface Pro tablet for the price?

op 4 :  Which is the better iPad Pro than Surface Pro?

op 5 :  What is the Surface Pro tetrad with and iPad Pro?

op 6 :  What is Surface Pro and how does it compare to iPad Pro?

op 7 :  Is Surface Pro an iPhone or an iPad Pro?

op 8 :  How does it compare to the iPad Pro?

op 9 :  How do you compare the Surface Pro to the iPad Pro?


------------------------------------------------------------

input:  Should I have a hair transplant at age 24? How much would it cost?

corrupted:  hair transplant age 24? a good deal would cost?

op 0 :  Would you be able[SEP]How much would a skin transplant procedure cost?

op 1 :  Is it a goo

In [77]:
original_2, corrupted_2, ans_2 = inference_for_given_sentence(args, gpt_model)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


START INFERENCE



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [78]:
for i, sentence in enumerate(original_2):
    print("input: ", sentence)
    
    print()
    print("corrupted: ", corrupted_2[i])
    print()
    for k, output in enumerate(ans_2[i]):
        print("op",k, ": ", output)
        print()
    print()
    print("------------------------------------------------------------")
    print()

input:  how do you send a private message to someone you’re following on quora?

corrupted:  send private message following quora ?

op 0 :  Can a private message being written to me be read by anyone on quora?

op 1 :  How can I start a private message to someone on Quora?

op 2 :  Can anyone to send me a private message following quora?

op 3 :  Can you follow me on here in path following traffic?

op 4 :  Can I type in a private message and then follow it

op 5 :  What do I do to block a follower who follows me with the private message following me?

op 6 :  Can I ask private messages on Quora?

op 7 :  How do I private message follow a private message on Quora?

op 8 :  Does quora follow your private message?

op 9 :  How can I follow seperate private messages on mequora?


------------------------------------------------------------

input:  do you believe donald trump can make america great again?

corrupted:  believe donald trump america great ?

op 0 :  Are Donald Trump or Bara

In [43]:
output_csv_path = "/kaggle/working/output2.csv"

with open(output_csv_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    
    # header row
    csvwriter.writerow(['Original Sentence', 'Paraphrase'])
    
    # Iterate over each original sentence and its paraphrases
    for i, sentence in enumerate(original_2):
        # Write each paraphrase with it's original
        for output in ans_2[i]:
            csvwriter.writerow([sentence, output])
            
print("Output saved to", output_csv_path)

Output saved to /kaggle/working/output2.csv


#### Evaluation

In [44]:
from transformers import BertTokenizer, BertModel
import torch

def calculate_contextual_similarity_hindi(sentence1, sentence2):
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
    model = BertModel.from_pretrained('bert-base-multilingual-cased')

    # Tokenize and encode sentences
    inputs_sentence1 = tokenizer(sentence1, return_tensors='pt', padding=True, truncation=True)
    inputs_sentence2 = tokenizer(sentence2, return_tensors='pt', padding=True, truncation=True)

    # Get BERT contexual embeddings for each sentences
    with torch.no_grad():
        outputs_sentence1 = model(**inputs_sentence1)
        outputs_sentence2 = model(**inputs_sentence2)

    # Calculate cosine sim between sentence embeddings
    similarity_score = torch.nn.functional.cosine_similarity(outputs_sentence1.last_hidden_state.mean(dim=1),
                                                             outputs_sentence2.last_hidden_state.mean(dim=1),
                                                             dim=1).item()
    return similarity_score

# Example 
sentence1 = " सपने वो नहीं होते जो हम सोते समय देखते हैं, सपने वो होते हैं जो हमें सोने नहीं देते।"
sentence2 = "नहीं सपने सपने हमें देते वो हमें सोने देते तो कैसे"
score = calculate_contextual_similarity_hindi(sentence1, sentence2)
print("Contextual similarity score between the Hindi sentences:", score)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Contextual similarity score between the Hindi sentences: 0.729651689529419


In [79]:
count = 0
li = []
for i, sentence in enumerate(original_2):
    temp = []
    count += 1
    if(count == 20):
        break
    print("input: ", sentence)
    print()
    print("corrupted: ", corrupted_2[i])
    print()
    for k, output in enumerate(ans_2[i]):
        
        score = calculate_contextual_similarity_hindi(sentence, output)
        temp.append((output, score))
        print("op",k, ": ", output, "score", score)
        print()
    print()
    print("------------------------------------------------------------")
    print()
    li.append(temp)

input:  how do you send a private message to someone you’re following on quora?

corrupted:  send private message following quora ?

op 0 :  Can a private message being written to me be read by anyone on quora? score 0.7600200176239014

op 1 :  How can I start a private message to someone on Quora? score 0.7959038019180298

op 2 :  Can anyone to send me a private message following quora? score 0.7663204669952393

op 3 :  Can you follow me on here in path following traffic? score 0.6384581923484802

op 4 :  Can I type in a private message and then follow it score 0.6570327877998352

op 5 :  What do I do to block a follower who follows me with the private message following me? score 0.7229054570198059

op 6 :  Can I ask private messages on Quora? score 0.7290664911270142

op 7 :  How do I private message follow a private message on Quora? score 0.7662537097930908

op 8 :  Does quora follow your private message? score 0.7435103058815002

op 9 :  How can I follow seperate private messages 

In [80]:
def get_score(item):
    return item[1]

# Print sorted list
for i, item in enumerate(li):
    item = sorted(item, key=get_score, reverse=True)
    print("input: ", original_2[i])
    print()
    print("corrupted: ", corrupted_2[i])
    print()
    for i, (output, score) in enumerate(item):
        print("Op",i + 1,": ", output, " Score:", score)
        print()
    print("------------------------------------------------------------")

input:  how do you send a private message to someone you’re following on quora?

corrupted:  send private message following quora ?

Op 1 :  How can I start a private message to someone on Quora?  Score: 0.7959038019180298

Op 2 :  Can anyone to send me a private message following quora?  Score: 0.7663204669952393

Op 3 :  How do I private message follow a private message on Quora?  Score: 0.7662537097930908

Op 4 :  Can a private message being written to me be read by anyone on quora?  Score: 0.7600200176239014

Op 5 :  Does quora follow your private message?  Score: 0.7435103058815002

Op 6 :  How can I follow seperate private messages on mequora?  Score: 0.7315950393676758

Op 7 :  Can I ask private messages on Quora?  Score: 0.7290664911270142

Op 8 :  What do I do to block a follower who follows me with the private message following me?  Score: 0.7229054570198059

Op 9 :  Can I type in a private message and then follow it  Score: 0.6570327877998352

Op 10 :  Can you follow me on h

#### Finished!!!